In [1]:
! pip install frictionless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.9/309.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.7/411.7 kB 12.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.3 MB/s eta 0:00:00
  Created wheel for petl: filename=petl-1.7.14-py3-none-any.whl size=229615 sha256=27318837472a2fa1ae31750ea16598cfc87d9c452dc612b7445d1c185a156625
  Stored in directory: /root/.cache/pip/wheels/65/d2/2c/655203aaf0c828a0b7481dfc3354814dd1b864f4c4b0a413a7
  Created wheel for stringcase: filename=stringcase-1.2.0-py3-none-any.whl size=3570 sh

In [2]:
import requests     # 2.18.4
import json         # 2.0.9
import pandas as pd # 0.23.0
import numpy as np
import time
import os
import frictionless
from getpass import getpass
from ftplib import FTP_TLS
from ftplib import FTP
import ftplib
import ast

In [7]:
import pandas as pd
import re
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup

url = 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/'

def extract_id(fileName):
    ogd_id = fileName.partition('_')[0]
    return ogd_id.replace('ogd', '')

def getInformation(url):

  csvList = []
  url = url.replace(" ","%20")
  req = Request(url)
  a = urlopen(req).read()
  soup = BeautifulSoup(a, 'html.parser')
  x = (soup.find_all('a'))

  for elem in x:
    stringElem = str(elem)

    if 'ogd' in stringElem:
      if 'csv' in stringElem:
        fileName = re.search('href="(.*)"', stringElem).group(1)
        csvList.append([fileName, extract_id(fileName)])

  df = pd.DataFrame(csvList, columns= ['fileName', 'ogd_id'])
  print(df)
  return df
  #df.to_csv('files/info.csv')

df = getInformation(url)

                                            fileName ogd_id
0  ogd35_schweizerische_elektrizitaetsbilanz_mona...     35
1                                     ogd39_epec.csv     39
2                     ogd47_vollzugsresultate_pw.csv     47
3                ogd47_vollzugsresultate_pw_2020.csv     47
4                    ogd61_vollzugsresultate_lnf.csv     61
5               ogd61_vollzugsresultate_lnf_2020.csv     61
6           ogd90_potenzialstudien_waermenutzung.csv     90


Test the adjusted files:

In [31]:
staging = 'staging/'
url_ogd = 'https://www.uvek-gis.admin.ch/BFE/ogd/'
url = 'https://www.uvek-gis.admin.ch/BFE/'
dataPackage = 'datapackage.json'

for index, row in df.iterrows():

  # load original data package
  folderPath = url_ogd + row['ogd_id'] + '/'
  datapackagePath = os.path.join(folderPath, dataPackage)

  # new file path --> to staging folder
  stagingFilePath = url_ogd + 'staging/' + row['fileName']

  # store the response of URL
  response = urlopen(datapackagePath)

  # storing the JSON response
  data_json = json.loads(response.read())

  #print('original datapackage: ')
  #print(data_json)


  # change source file
  jsonAsString = str(data_json)
  jsonAsString = jsonAsString.replace(os.path.join(folderPath, row['fileName']), stagingFilePath)
  updatedJSON = ast.literal_eval(jsonAsString)

  #print('---------------------------')
  #print('adjusted datapackage: ')
  #print(updatedJSON)
  #print('===========================')

  # path for temporary datapackage
  tmpJSON_path = 'tmpPackage.json'

  # save the file as data.json
  with open(tmpJSON_path, 'w') as f:
    json.dump(updatedJSON, f)

  report = frictionless.validate(tmpJSON_path)

  if report.valid:
    #print("Valid.\n")
    k = 2
  else:
    print("Not valid!\n")
    print(len(report.tasks))
    print()
    print()

    for err in range(len(report.tasks)):
      print(report.tasks[0].errors[err])
      print('---------------------------')

  print('===========================')
  print()






Not valid!

2


{'type': 'type-error',
 'title': 'Type Error',
 'description': 'The value does not match the schema type and format for this '
                'field.',
 'message': 'Type error in the cell "HALLO2" in row "13" and field '
            '"sanktion_final" at position "10": type is "number/default"',
 'tags': ['#table', '#row', '#cell'],
 'note': 'type is "number/default"',
 'cells': ['2021',
           '654655 AG',
           'Einzelabrechnung',
           '1',
           '1848',
           '175',
           '175',
           '125.1',
           '49.9',
           'HALLO2',
           '0',
           '9.3',
           '0'],
 'rowNumber': 13,
 'cell': 'HALLO2',
 'fieldName': 'sanktion_final',
 'fieldNumber': 10}
---------------------------
{'type': 'type-error',
 'title': 'Type Error',
 'description': 'The value does not match the schema type and format for this '
                'field.',
 'message': 'Type error in the cell "HALLO" in row "14" and field "anteil_lev" '
   